In [1]:
library("forecast")
library("stringr")
library("ggplot2")
library("MLmetrics")

Warning message:
"package 'MLmetrics' was built under R version 3.5.2"
Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall



In [2]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\data\\Mexico\\processed_data")

In [3]:
files = list.files(".", pattern="*.csv")
files

[1] "Chiapas_2016-2017.csv"   "Guerrero_2016-2017.csv" 
[3] "NuevoLeon_2016-2017.csv" "Veracruz_2016-2017.csv" 
[5] "Yucatan_2016-2017.csv"

In [4]:
myfiles = lapply(files, function(i){
    read.csv(i, header=TRUE)
})
print(myfiles)

[[1]]
          Date Searches Cases
1   02/01/2016      100     9
2   09/01/2016      100     0
3   16/01/2016      100     3
4   23/01/2016       55    11
5   30/01/2016       98    11
6   06/02/2016       89    10
7   13/02/2016       70     9
8   20/02/2016       38     3
9   27/02/2016       58    15
10  05/03/2016       67     3
11  12/03/2016       81     7
12  19/03/2016      100     6
13  26/03/2016       65    14
14  02/04/2016       45    10
15  09/04/2016       67    11
16  16/04/2016      100     1
17  23/04/2016       99     5
18  30/04/2016       73     3
19  07/05/2016      100     0
20  14/05/2016      100     1
21  21/05/2016      100     0
22  28/05/2016      100    10
23  04/06/2016      100     7
24  11/06/2016      100     0
25  18/06/2016      100    59
26  25/06/2016      100    43
27  02/07/2016       59    45
28  09/07/2016       44    55
29  16/07/2016      100    45
30  23/07/2016       75    46
31  30/07/2016       64    21
32  06/08/2016       34    20
33  

In [5]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\Mexico\\SARIMA\\1Week")

In [6]:
myplot <- function(dataset, filename) {
    print("-----")
    print(filename)
    split_index = nrow(dataset)/2
    
    window_size = 4
    lag_weeks = 1

    train = dataset[1:(split_index - lag_weeks + 1),2:3]
    test = dataset[(split_index+1):(nrow(dataset)), 2:3]

    endIndex = nrow(test)
    trainIndex = nrow(train)
    
    predictions = numeric(0)

    for (index in 1:endIndex){
        fit <- auto.arima(dataset$Cases[1:trainIndex+index-1], seasonal=TRUE, xreg=dataset$Searches[1:trainIndex+index-1])
        
        #Forecast regressors
        fit_reg = auto.arima(dataset$Searches[1:trainIndex+index-1])
        reg_forecast <- forecast(fit_reg, h=lag_weeks)
        reg_forecast_frame <- data.frame(reg_forecast)
        reg_forecast_point_forecast <- reg_forecast_frame$Point.Forecast
        reg_df = dataset$Searches[1:trainIndex+index-1]
        l = 1
        reg_length = length(reg_df)
        
        while(l <= lag_weeks) {
            reg_df[reg_length + l] = reg_df[l]
            l = l + 1
        }
        
        #Forecast
        
#         print(arimaorder(fit))
        fcast <- forecast(fit, h=lag_weeks, xreg = reg_df)
        forecasted_data <- data.frame(fcast)
        predictions[index] <- forecasted_data$Point.Forecast[lag_weeks]
    }
    
    length(predictions)
    print(filename)
#     output = cat("1Week", filename)
    write.csv(predictions, file=filename)
    mse = MSE(predictions, test$Cases)
    print(mse)
    
}

In [7]:
fileIndex = 1
for(file in myfiles) {
    myplot(file, files[fileIndex])
    fileIndex = fileIndex + 1
}

[1] "-----"
[1] "Chiapas_2016-2017.csv"
[1] "Chiapas_2016-2017.csv"
[1] 42.09098
[1] "-----"
[1] "Guerrero_2016-2017.csv"
[1] "Guerrero_2016-2017.csv"
[1] 7.58667
[1] "-----"
[1] "NuevoLeon_2016-2017.csv"
[1] "NuevoLeon_2016-2017.csv"
[1] 353.594
[1] "-----"
[1] "Veracruz_2016-2017.csv"
[1] "Veracruz_2016-2017.csv"
[1] 211.5843
[1] "-----"
[1] "Yucatan_2016-2017.csv"
[1] "Yucatan_2016-2017.csv"
[1] 278.4354
